In [11]:
import tensorflow as tf
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import tf_py_environment
from tf_agents.environments import gym_wrapper
import tf_agents
from tf_agents.utils import common
from tf_agents.policies import random_tf_policy
from tf_agents.specs import tensor_spec
from tf_agents.policies import random_tf_policy
from tf_agents.specs import tensor_spec
from tf_agents.replay_buffers import py_uniform_replay_buffer
import numpy as np
import pandas as pd
import datetime
# import gymnasium as gym
import gym
import numpy as np
import random
import MetaTrader5 as mt5
from collections import deque

In [12]:
def get_data(currency,):
    mt5.initialize()
    ohlc_data = pd.DataFrame(mt5.copy_rates_range(currency, mt5.TIMEFRAME_M15, datetime.datetime(2015, 1, 1), datetime.datetime.now()))
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data
def splitdf(data,size):
    splitdata=np.array_split(data, size)
    return splitdata

In [4]:
dat=get_data('GOLD')
dfs=splitdf(dat,2)
dataf=dfs[0]

df = dataf.iloc[:1000]

eval_data = dataf.iloc[1000:2000]

c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
df.drop(columns=['time'], inplace=True)
df.drop(columns=['tick_volume'], inplace=True)
df.drop(columns=['spread'], inplace=True)
df.drop(columns=['real_volume'], inplace=True)
df

C:\Users\DELL\AppData\Local\Temp\ipykernel_11428\2474045239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['time'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11428\2474045239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['tick_volume'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11428\2474045239.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=

,Open,High,Low,Close
0,1186.07,1187.03,1186.07,1187.03
1,1187.03,1187.03,1186.53,1186.83
2,1186.83,1187.23,1186.03,1186.34
3,1186.35,1186.99,1186.35,1186.79
4,1186.94,1187.04,1185.43,1185.53
...,...,...,...,...
995,1277.63,1278.53,1276.69,1278.53
996,1278.63,1278.93,1277.59,1278.48
997,1278.37,1278.93,1277.73,1278.82
998,1278.77,1279.02,1277.22,1277.52


In [6]:
eval_data.drop(columns=['time'], inplace=True)
eval_data.drop(columns=['tick_volume'], inplace=True)
eval_data.drop(columns=['spread'], inplace=True)
eval_data.drop(columns=['real_volume'], inplace=True)
eval_data

C:\Users\DELL\AppData\Local\Temp\ipykernel_11428\218666643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data.drop(columns=['time'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11428\218666643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data.drop(columns=['tick_volume'], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11428\218666643.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_

,Open,High,Low,Close
1000,1278.27,1278.61,1276.31,1277.40
1001,1277.30,1278.29,1275.96,1276.41
1002,1276.41,1276.64,1275.33,1276.44
1003,1276.49,1276.89,1275.32,1275.44
1004,1275.39,1277.53,1274.89,1277.23
...,...,...,...,...
1995,1275.00,1275.20,1273.57,1273.79
1996,1273.50,1273.98,1272.63,1273.75
1997,1274.41,1275.35,1274.25,1274.60
1998,1274.60,1274.90,1274.50,1274.65


In [53]:
u=np.array([2,3,4])
u=np.array(u)
u

array([2, 3, 4])

In [7]:
ohlc_data = df[['Open', 'High', 'Low', 'Close']].values
ohlc_data

array([[1186.07, 1187.03, 1186.07, 1187.03],
       [1187.03, 1187.03, 1186.53, 1186.83],
       [1186.83, 1187.23, 1186.03, 1186.34],
       ...,
       [1278.37, 1278.93, 1277.73, 1278.82],
       [1278.77, 1279.02, 1277.22, 1277.52],
       [1277.42, 1279.01, 1277.42, 1278.21]])

In [51]:
tb=gym.spaces.Box(low=0, high=1, shape=(1,4), dtype=np.float32)
tb

Box(0.0, 1.0, (1, 4), float32)

In [82]:

class finalv(gym.Env):
    def __init__(self, initial_balance, data):
        super(finalv, self).__init__()

        # Define the OHLC data
        self.ohlc_data = data[['Open', 'High', 'Low', 'Close']].values
        self.current_index = 0
        self.current_trade = 0
        self.current_balance = initial_balance
        self.sl=0
        self.tp=0
        self.total_profit = 0.0
        
        self.entry=0
        

        # Define the observation space (using close prices only for simplicity)
        self.observation_space = gym.spaces.Box(low=0, high=np.max(ohlc_data[:,1]-ohlc_data[:,2]), shape=(12,))
        self.action_space = gym.spaces.Discrete(3)
        

    def count_profit(self, tradetype, entry, price):
        pr = price - entry
        if tradetype == 1:
            return price-entry
        elif tradetype == -1:
            return entry-price

    def _process_trade(self, profit):
        self.current_balance += profit
        self.total_profit += profit
    
    
        
   

    def step(self, action):
        reward = 0
        if self.current_trade==0:
            if action == 0:
                reward = 0
            elif action == 1:
                self.current_trade = 1
                self.entry = self.ohlc_data[self.current_index][3]
               
                reward = 0
            elif action == 2:
                self.current_trade = -1
                self.entry = self.ohlc_data[self.current_index][3]
                
                reward = 0
        elif self.current_trade==1:
            if action == 0:
                reward = 0
            elif action == 1:
                self.current_trade =0
                self._process_trade(self.count_profit(1, self.entry, self.ohlc_data[self.current_index][3]))
                reward = self.count_profit(1, self.entry, self.ohlc_data[self.current_index][3])
            elif action == 2:
                reward = 0
        elif self.current_trade==-1:
            if action == 0:
                reward = 0
            elif action == 1:
                
                self.current_trade = 0
                self._process_trade(self.count_profit(-1, self.entry, self.ohlc_data[self.current_index][3]))
                reward = self.count_profit(-1, self.entry, self.ohlc_data[self.current_index][3])
                
            elif action == 2:
               reward=0
            
                
           
            

        done = None
        if self.current_index >= len(self.ohlc_data) - 1:
          done=True
        


        prev5_items=self.ohlc_data[self.current_index-4: self.current_index+1]
        prevcandle_spread=prev5_items[:,1]-prev5_items[:,2]
        prevcandle_bodysizes=prev5_items[:,3]-prev5_items[:,0]
        candletype= -1 if self.ohlc_data[self.current_index][3]-self.ohlc_data[self.current_index][0] < 0 else 1
        istrade=self.current_trade
        state=np.concatenate((prevcandle_spread,prevcandle_bodysizes))
        state=np.insert(state,0,istrade)
        state=np.insert(state,0,candletype)
        state=np.array(state)
        
        discount = 0.99  # Set your desired discount factor
        step_type = 0  # Set the appropriate value for step_type

        # Construct the time step as a dictionary
        time_step = {'step_type': step_type, 'reward': reward, 'discount': discount, 'observation': state}
        info = {}
        if not done:
          self.current_index += 1
        return state, reward, done, info

    def render(self, mode='human'):
        if mode == 'human':
            print(f"Total Cumulative Profit: {self.total_profit}")
        else:
            raise NotImplementedError("Unsupported rendering mode")

    def reset(self):
        self.current_index = 6
        self.current_trade = 0
        self.total_profit = 0.0
        self.current_balance = 0
        return np.array([0,0,0,0,0,0,0,0,0,0,0,0])



In [83]:
envn =finalv(100,df)

In [84]:

train = gym_wrapper.GymWrapper(envn)
train_env = tf_py_environment.TFPyEnvironment(train)


In [85]:
train_env.time_step_spec().observation


BoundedTensorSpec(shape=(12,), dtype=tf.float32, name='observation', minimum=array(0., dtype=float32), maximum=array(17.33, dtype=float32))

In [86]:
action = np.array(1, dtype=np.int32)
print(action)
train_env.step(action)

1


TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 12), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>})

In [87]:

time_step = train_env.reset()
print(time_step)
while not time_step.is_last():
  time_step = train_env.step(action)
  print(time_step)

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 12), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>})
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[ 1.  ,  1.  ,  1.2 ,  0.64,  1.61,  1.1 ,  1.5 , -0.49,  0.44,
        -1.41, -0.3 ,  0.78]], dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>})
TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[-

In [46]:
print(type(action))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [88]:

tf_env = train_env

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 1

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    # print (type(action))
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)

num_episodes: 1 num_steps: 994
avg_length 994.0 avg_reward: 69.659996


In [89]:
eval_envn =finalv(100,eval_data)
eval = gym_wrapper.GymWrapper(eval_envn)
eval_env = tf_py_environment.TFPyEnvironment(eval)



In [90]:
fc_layer_params = (100,)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)


In [101]:
from tensorflow.keras.optimizers.legacy import Adam


In [102]:
optimizer = Adam(learning_rate=1e-3)

train_step_counter = tf.Variable(0)

agent00 = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent00.initialize()

In [105]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),train_env.action_spec())



In [106]:
def compute_avg_return(environment, policy, num_episodes):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]



In [107]:
compute_avg_return(train_env,random_policy,6)

-11.241664

In [108]:
replay_buffer_capacity = 1000*32 # same capacity as the TFUniformReplayBuffer

py_replay_buffer = py_uniform_replay_buffer.PyUniformReplayBuffer(
    capacity=replay_buffer_capacity,
    data_spec=tensor_spec.to_nest_array_spec(agent00.collect_data_spec))

In [109]:
dataset = py_replay_buffer.as_dataset(
    sample_batch_size=170,
    num_steps=2)

iterator = iter(dataset)

In [110]:
agent00.train = common.function(agent00.train)
tr=next(iterator)
print(tr)
y=agent00.train(experience=tr)
y

Trajectory(
{'action': <tf.Tensor: shape=(170, 2), dtype=int64, numpy=
array([[                  0, 4607182418800017408],
       [                  0,                   2],
       [9218868437227405312, 4607182418800017408],
       [      2545320673392,       2542620639235],
       [      2545908922944,       2542620639233],
       [      2542620639233,       2542620639233],
       [      2545320673392,       2542620639235],
       [      2545985472576,       2542620639233],
       [      2542620639233,       2542620639233],
       [      2542620639233,       2542620639233],
       [      2542620639233,       2542620639233],
       [      2545985472576,       2542620639233],
       [      2542620639233,       2545320673392],
       [      2542620639235,       2542620639233],
       [      2545941115200,       2545941115200],
       [      2542620639235,       2542620639233],
       [      2545909720320,       2542620639233],
       [      2545924426304,       2542620639233],
       [   

InvalidArgumentError: Graph execution error:

Detected at node 'loss/GatherNd' defined at (most recent call last):
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\asynci

In [97]:
agent00.train = common.function(agent00.train)
agent00.train_step_counter.assign(0)
Btrain_eval=6
avg_return = compute_avg_return(eval_env, agent00.policy,Btrain_eval)
returns = [avg_return]
time_step = train_env.reset()
num_train_steps = 200
eval_interval=5
log_interval=15

for _ in range(num_train_steps):
  
  trajectories= next(iterator)
  loss = agent00.train(experience=trajectories).loss
  step = agent00.train_step_counter.numpy()
  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step,loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent00.policy, Btrain_eval)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)
  print(f"Step: {_}, Loss: {loss}")

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


InvalidArgumentError: Graph execution error:

Detected at node 'loss/GatherNd' defined at (most recent call last):
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\DELL\anaconda3\envs\agent2\lib\asynci